# 4. Model Training with Second-Order Optimizers

We can use second-order optimizers to improve the training precision and potentially get better fit results. 

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
from m3util.viz.printing import printer
from m3util.viz.style import set_style
from m3util.ml.rand import set_seeds
from m3util.ml.optimizers.AdaHessian import AdaHessian
from m3util.ml.optimizers.TrustRegion import TRCG
from m3util.util.IO import download_and_unzip, make_folder
from belearn.dataset.dataset import BE_Dataset
from belearn.viz.viz import Viz
from belearn.functions.sho import SHO_nn
from autophyslearn.postprocessing.complex import ComplexPostProcessor
from autophyslearn.spectroscopic.nn import Multiscale1DFitter, Model


printing = printer(basepath = './Figures/')

set_style("printing")
set_seeds(seed=42)

%matplotlib inline

## Loads Data


In [ ]:
# Specify the filename and the path to save the file
filename = "./data_raw.h5"
save_path = "./Data"


data_path = save_path + "/" + filename

# instantiate the dataset object
dataset = BE_Dataset(data_path, SHO_fit_func_LSQF=SHO_nn)

# print the contents of the file
dataset.print_be_tree()

In [ ]:
# instantiates the visualization object
BE_viz = Viz(dataset, printing, verbose=True)

## ADAM model

In [ ]:
# constructs a test train split
X_train, X_test, y_train, y_test = dataset.test_train_split_(shuffle=True)

set_seeds(seed=42)

postprocessor = ComplexPostProcessor(dataset)

ADAM_model_ = Multiscale1DFitter(SHO_nn, # function 
                            dataset.frequency_bin, # x data
                            2, # input channels
                            4, # output channels
                            dataset.SHO_scaler, 
                            postprocessor)

# instantiate the model
ADAM_model = Model(ADAM_model_, dataset, 
                   training=True, model_basename="SHO_Fitter_original_data", 
                   datafed_path = "2024_SHO_Fitting/Adam_NN_SHO_Fitter", 
                   script_path = f"{os.getcwd()}/4_Second_Order_Optimizers.ipynb")

ADAM_model.fit(
    dataset.X_train,
    5000,
    optimizer="Adam",
    epochs = 5,
)



## Adahessian

We first try an optimized called [adahessian](https://arxiv.org/abs/2006.00719)

In [ ]:
# constructs a test train split
X_train, X_test, y_train, y_test = dataset.test_train_split_(shuffle=True)

set_seeds(seed=42)

postprocessor = ComplexPostProcessor(dataset)

AdaHessian_ = Multiscale1DFitter(SHO_nn, # function 
                            dataset.frequency_bin, # x data
                            2, # input channels
                            4, # output channels
                            dataset.SHO_scaler, 
                            postprocessor)

# instantiate the model
AdaHessian_model = Model(AdaHessian_, dataset, training=True, 
                         model_basename="SHO_Fitter_original_data_adahessian",
                        datafed_path = "2024_SHO_Fitting/AdaHessian_NN_SHO_Fitter", 
                       script_path = f"{os.getcwd()}/4_Second_Order_Optimizers.ipynb")


AdaHessian_model.fit(
    dataset.X_train,
    5000,
    optimizer="AdaHessian",
    epochs = 5,
     lr=.15
)


### Evaluate the Neural Network Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by comparing the different stages of the autoencoder including the raw data, the generated curves.


In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_train,
    prediction=AdaHessian_model,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename="Figure_4_1_NN_Adahessian_SHO_Fits_training",
)

**Figure 4.1** Raw data and reconstruction of the autoencoder trained with Adahessian for the training data. Shows the a) best, b) median, and c) worst reconstruction of the training data.

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_test,
    prediction=AdaHessian_model,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename="Figure_4_2_NN_Adahessian_SHO_Fits_testing",
)

**Figure 4.2** Raw data and reconstruction of the autoencoder trained with Adahessian for the testing data. Shows the a) best, b) median, and c) worst reconstruction of the testing data.

In [ ]:
state = {
    "fitter": "LSQF",
    "raw_format": "complex",
    "resampled": True,
    "scaled": True,
    "output_shape": "index",
}

X_data, Y_data = dataset.NN_data()

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    state,
    prediction=AdaHessian_model,
    out_state={"scaled": True, "measurement State": "complex"},
    returns=True,
    filename="Figure_4_3_NN_Adahessian_SHO_Fits_testing",
)

**Figure 4.3** Raw data and reconstruction of the autoencoder trained with Adahessian for the entire data. Shows the a) best, b) median, and c) worst reconstruction of the entire data.

In [ ]:
# we will add the appropriate phase shift to the dataset based on the fix seed,
# If your seed is different the results might vary
dataset.NN_phase_shift = np.pi/2 

# you can view the test and training dataset by replacing X_data with X_test or X_train
pred_data, scaled_param, parm = AdaHessian_model.predict(X_data)

BE_viz.SHO_hist(parm, filename="Figure_13_NN_Unscaled_Parameters_Histograms")

**Figure 4.4** Histogram of the fit results computed with the Adahessian optimized neural network for the a) amplitude, b) resonance frequency, c) quality factor, and d) phase.

In [ ]:
# you can view the test and training dataset by replacing X_data with X_test or X_train
pred_data, scaled_param, parm = AdaHessian_model.predict(X_data)

BE_viz.SHO_switching_maps(parm, filename="Figure_3_5_NN_Switching_Maps")

**Figure 4.5** Snapshots of the amplitude, resonance frequency, quality factor, and phase of the piezoresponse during ferroelectric switching based on adahessian optimizer fits.

In [ ]:
dataset.NN_phase_shift = np.pi/2
dataset.LSQF_phase_shift = np.pi/2
dataset.measurement_state = "all"


true_state = {
    "fitter": "LSQF",
    "raw_format": "complex",
    "resampled": True,
    "scaled": True,
    "output_shape": "index",
    "measurement_state": "all",
}

out_state = {"scaled": True, "raw_format": "magnitude spectrum"}

n = 1

LSQF = BE_viz.get_best_median_worst(
    true_state,
    prediction={"fitter": "LSQF"},
    out_state=out_state,
    SHO_results=True,
    n=n,
)

NN = BE_viz.get_best_median_worst(
    true_state, prediction=AdaHessian_model, out_state=out_state, SHO_results=True, n=n
)

data = (LSQF, NN)
names = ["LSQF", "NN"]

BE_viz.SHO_Fit_comparison(
    data,
    names,
    model_comparison=[AdaHessian_model, {"fitter": "LSQF"}],
    out_state=out_state,
    filename="Figure_4_5_LSQF_NN_bmw_comparison",
)

**Figure 4.6** Comparison of the a,b) best, c,d) median, and e,f) worst simple harmonic oscillator fits. a,c,e) are based on the fit results from the simple harmonic oscillator model, wherein b,d,f) are based on the neural network model trained with the Adahessian optimizer. Generally speaking, all fits are good for both models on the as collected data.

In [ ]:
true_state = {
    "fitter": "LSQF",
    "raw_format": "complex",
    "resampled": True,
    "scaled": True,
    "output_shape": "index",
    "measurement_state": "all",
}

BE_viz.violin_plot_comparison_SHO(true_state, AdaHessian_model, X_data, label='AdaHessian',
                                  filename="Figure_4_7_Violin_plot_comparison_SHO_AdaHessian") 

**Figure 4.7** Comparison of the least-squares and adahessian optimized neural network fitting results.

## Trust Region Optimizer

In [ ]:
optimizer = {"name": "TRCG", "optimizer":TRCG, "radius": 5, "device": "cuda","ADAM_epochs": 2}

# constructs a test train split
X_train, X_test, y_train, y_test = dataset.test_train_split_(shuffle=True)

set_seeds(seed=42)

postprocessor = ComplexPostProcessor(dataset)

TRCG_ = Multiscale1DFitter(SHO_nn, # function 
                            dataset.frequency_bin, # x data
                            2, # input channels
                            4, # output channels
                            dataset.SHO_scaler, 
                            postprocessor)

# instantiate the model
TRCG_model = Model(TRCG_, dataset, training=True, model_basename="SHO_Fitter_original_data_TRCG",
                   datafed_path = "2024_SHO_Fitting/TR_NN_SHO_Fitter", 
                   script_path = f"{os.getcwd()}/4_Second_Order_Optimizers.ipynb")

TRCG_model.fit(
    dataset.X_train,
    5000,
    optimizer=optimizer,
    epochs = 5,
)

### Evaluate the Neural Network Results

It is always recommended to validate that the autoencoder is working correctly. We can do this by comparing the different stages of the autoencoder including the raw data, the generated curves.


In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_train,
    prediction=TRCG_model,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename="Figure_4_8_NN_TrustRegion_SHO_Fits_training",
)

**Figure 4.8** Raw data and reconstruction of the autoencoder trained with Trust Region for the training data. Shows the a) best, b) median, and c) worst reconstruction of the training data.

In [ ]:
d1, d2, index1, mse1 = BE_viz.bmw_nn(
    X_test,
    prediction=TRCG_model,
    out_state={"scaled": True, "raw_format": "complex"},
    returns=True,
    filename="Figure_4_9_NN_TrustRegion_SHO_Fits_testing",
)

**Figure 4.9** Raw data and reconstruction of the autoencoder trained with trust region for the testing data. Shows the a) best, b) median, and c) worst reconstruction of the testing data.

In [ ]:
state = {
    "fitter": "LSQF",
    "raw_format": "complex",
    "resampled": True,
    "scaled": True,
    "output_shape": "index",
}

X_data, Y_data = dataset.NN_data()

d1, d2, index1, mse1 = BE_viz.bmw_nn(
    state,
    prediction=TRCG_model,
    out_state={"scaled": True, "measurement State": "complex"},
    returns=True,
    filename="Figure_4_10_NN_validation_full_data",
)

**Figure 4.10** Raw data and reconstruction of the autoencoder trained with Adahessian for the entire data. Shows the a) best, b) median, and c) worst reconstruction of the entire data.

In [ ]:
# we will add the appropriate phase shift to the dataset based on the fix seed,
# If your seed is different the results might vary
dataset.NN_phase_shift = np.pi/2 

# you can view the test and training dataset by replacing X_data with X_test or X_train
pred_data, scaled_param, parm = TRCG_model.predict(X_data)

BE_viz.SHO_hist(parm, filename="Figure_4_11_NN_Unscaled_Parameters_Histograms")

**Figure 4.11** Histogram of the fit results computed with the trust region optimized neural network for the a) amplitude, b) resonance frequency, c) quality factor, and d) phase.

In [ ]:
# you can view the test and training dataset by replacing X_data with X_test or X_train
pred_data, scaled_param, parm = TRCG_model.predict(X_data)

BE_viz.SHO_switching_maps(parm, filename="Figure_4_12_NN_Switching_Maps")

**Figure 4.12** Snapshots of the amplitude, resonance frequency, quality factor, and phase of the piezoresponse during ferroelectric switching based on the trust region optimizer fits.

In [ ]:
dataset.NN_phase_shift = np.pi/2
dataset.LSQF_phase_shift = np.pi/2
dataset.measurement_state = "all"


true_state = {
    "fitter": "LSQF",
    "raw_format": "complex",
    "resampled": True,
    "scaled": True,
    "output_shape": "index",
    "measurement_state": "all",
}

out_state = {"scaled": True, "raw_format": "magnitude spectrum"}

n = 1

LSQF = BE_viz.get_best_median_worst(
    true_state,
    prediction={"fitter": "LSQF"},
    out_state=out_state,
    SHO_results=True,
    n=n,
)

NN = BE_viz.get_best_median_worst(
    true_state, prediction=TRCG_model, out_state=out_state, SHO_results=True, n=n
)

data = (LSQF, NN)
names = ["LSQF", "NN"]

BE_viz.SHO_Fit_comparison(
    data,
    names,
    model_comparison=[TRCG_model, {"fitter": "LSQF"}],
    out_state=out_state,
    filename="Figure_4_13_LSQF_NN_bmw_comparison_Trust_Region",
    # display_results = None
)

**Figure 4.13:** Comparison of the a, b) best, c, d) median, and e, f) worst simple harmonic oscillator fits. a, c, e) are based on the fit results from the simple harmonic oscillator model, whereas b, d, f) are based on the neural network model trained with the Trust Region Optimizer. Generally speaking, all fits are good for both models on the collected data. The Trust Region Optimizer has lower maximum errors than the other neural network models.

In [ ]:
true_state = {
    "fitter": "LSQF",
    "raw_format": "complex",
    "resampled": True,
    "scaled": True,
    "output_shape": "index",
    "measurement_state": "all",
}


BE_viz.violin_plot_comparison_SHO(true_state, TRCG_model, X_data, filename="Figure_4_14_Violin_Trust_Region", label="Trust Region") 

**Figure 4.14** Comparison of the least-squares and trust region-optimized neural network fitting results.